In [ ]:
import deeplabcut as dlc
import random
import shutil
import glob
import os

# From paths.py
import paths

In [ ]:
# Find all videos in the data directory with VTuning in the name
all_videos = glob.glob(paths.vrexperiment_path + os.sep + "*VWheel*.avi")

In [ ]:
working_dir = r"D:\DLC_projects"
project_name = "vwheel_pupil_modelzoo"
experimenter = "matt"

# If Project already created, run the cell below

In [ ]:
config_path = glob.glob(os.path.join(working_dir, project_name) + "*/config.yaml")[0]
cfg = dlc.auxiliaryfunctions.read_config(config_path)
project_directory = cfg['project_path']
project_video_list = [vid for vid in cfg["video_sets"]]
# test_vids = os.path.join(project_video_list, "test-videos")

# If you haven't created a new project, run this cell

In [ ]:
# Randomly select videos to train on
remote_video_list = []
for i in range(2):
    remote_video_list.append(random.choice(all_videos))

In [ ]:
dlc.create_pretrained_project?

In [ ]:
config_path, train_config_path = dlc.create_pretrained_project(
    project_name,
    experimenter,
    remote_video_list,
    model="mouse_pupil_vclose",
    working_directory=working_dir,
    analyzevideo=False,
    createlabeledvideo=False,
    copy_videos=True, #must leave copy_videos=True
)

config_path = glob.glob(os.path.join(working_dir, project_name) + "*/config.yaml")[0]
cfg = dlc.auxiliaryfunctions.read_config(config_path)
project_directory = cfg['project_path']
project_video_list = [vid for vid in cfg["video_sets"]]

In [ ]:
# Update some config paramters
# Updating the plotting within the config.yaml file (without opening it ;):
edits = {
    'dotsize': 3,  # size of the dots!
    'colormap': 'jet',  # any matplotlib colormap!
    'pcutoff': 0.85,  # the higher the more conservative the plotting!
}
dlc.auxiliaryfunctions.edit_config(config_path, edits)

# Extract and label frames

In [ ]:
dlc.extract_frames?

In [ ]:
# extract frames from the videos
# will go through all of the videos on the list, so if many videos, it might take a while
dlc.extract_frames(config_path, mode='automatic', algo='kmeans', crop=False, userfeedback=False);

In [ ]:
dlc.label_frames?

In [ ]:
# label frames
# generates folders in the labeled-data folder with the video name
dlc.label_frames(config_path)

In [ ]:
dlc.check_labels?

In [ ]:
# check the labels
# generates folders in the labeled-data folder with the video name but the actual labels printed on the pics
dlc.check_labels(config_path, draw_skeleton=False, visualizeindividuals=False);

In [ ]:
dlc.create_training_dataset?

In [ ]:
# create the training set
# also creates the Pose.yaml in the dlc-models/train dataset, which contains the setting for training, i.e. take a look
dlc.create_training_dataset(config_path, augmenter_type='imgaug');

In [ ]:
# train the network
dlc.train_network(config_path, gputouse=0, allow_growth=True,
                  max_snapshots_to_keep=10, displayiters=5e3, saveiters=50e3)

In [ ]:
# evaluate the network
dlc.evaluate_network(config_path, Shuffles=[1], plotting=True)

In [ ]:
dlc.analyze_videos?

In [ ]:
dlc.analyze_videos(config_path, [r"D:\DLC_projects\vwheel_pupil_modelzoo-matt-2022-11-25\videos\test-short.mp4"], dynamic=(True, .1, 50), gputouse=0)

In [ ]:
dlc.create_labeled_video(config_path, [r"D:\DLC_projects\vwheel_pupil_modelzoo-matt-2022-11-25\videos\test-short.mp4"], fastmode=True)

In [ ]:
# extract outlier frames
# dlc.extract_outlier_frames(config_path, project_video_list, outlieralgorithm='uncertain', p_bound=0.6)
dlc.extract_outlier_frames(config_path, [r"D:\DLC_projects\vwheel_pupil_modelzoo-matt-2022-11-25\videos\test-short.mp4"])

In [ ]:
dlc.refine_labels(config_path)

In [ ]:
# Merge the dataset, then move to check labels and retrain
dlc.merge_datasets(config_path)

# Add extra videos (if needed)

In [ ]:
# If adding new videos, do that here by randomly selecting some not in the original data set or the test set
# For this particular case, we want videos that are older than March 4th 2022
from datetime import datetime
from os.path import sep
valid_extras = [vid for vid in all_videos if datetime.strptime(vid.split(sep)[-1].split('_VWheel')[0], "%m_%d_%Y_%H_%M_%S") > datetime(2022, 3, 5)]

extra_vids = []
while(len(extra_vids) < 5):
    vid = random.choice(valid_extras)
    if vid not in project_video_list or test_vids:
        extra_vids.append(vid)
        
dlc.add_new_videos(config_path, extra_vids, copy_videos=True)